# Aprendizaje Multietiqueta de Patrones Geométricos en Objetos de Herencia Cultural
# C2AE Training
## Seminario de Tesis II, Primavera 2022
### Master of Data Science. Universidad de Chile.
#### Prof. guía: Benjamín Bustos - Prof. coguía: Iván Sipirán
#### Autor: Matías Vergara

El objetivo de este notebook es realizar el etiquetado múltiple de patrones geométricos mediante C2AE.
El código se basa en la implementación del usuario ssmele en github: https://github.com/ssmele/C2AEinTorch.git

## Imports

In [1]:
root_dir = '..'

In [2]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from PIL import Image
import math
import pickle

from C2AE import C2AE, save_model, load_model, Fe, Fx, Fd, eval_metrics, get_predictions

from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, DataLoader

import os

from textwrap import wrap


from utils import KunischMetrics
from utils import KunischPruner
from utils import DataExplorer
from utils import KunischPlotter

## Configuración de dispositivo

In [3]:
# 0 es 3090, 1 y 2 son 2080
CUDA_ID = 0

device = torch.device(f'cuda:{CUDA_ID}' if torch.cuda.is_available() else 'cpu')
print(f"Usando device: {torch.cuda.get_device_name(device)}")

Usando device: NVIDIA GeForce RTX 3090


## Configuración de experimento

In [4]:
# Flags para los datos sintéticos
# Cada flag está asociada a una o más funciones de data augmentation.
# Los datos deben existir previamente 
# (se generan a partir del notebook split and augmentation)
DS_FLAGS = []
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'gausblur': [gausblur]
              # 'msblur': [msblur]
              # 'mtnblur': [mtnblur]
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
            
# Las flags crop, randaug, elastic y gausblur 
# se pueden aplicar más de una vez c/u. 
# (si no están en DS_FLAGS, serán ignoradas).
CROP_TIMES = 1
RANDOM_TIMES = 1
ELASTIC_TIMES = 1
GAUSBLUR_TIMES = 1

NUM_LABELS = 26
BATCH_SIZE = 100
PATIENCE = 100
NUM_EPOCHS = 600
FEATURES_DIM = 4096

# C2AE Scene config
latent_dim = 70
num_labels = NUM_LABELS
fx_h_dim= 120
fe_h_dim= 120
fd_h_dim= 120

# 0 es 3090, 1 y 2 son 2080
CUDA_ID = 0

SAVE = True
K = 4

In [5]:
# Esta celda construye la variable data_flags, que lee DS_FLAGS de 
# la celda anterior y mapea su contenido a distintas rutas de 
# patrones, etiquetas y outputs
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
         'gausblur': GAUSBLUR_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic', 'gausblur']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

# Revisión de los folds y creación de diccionario con paths
Kfolds = {}


for i in range(0, K):
    print("Fold ", i)
    
    exp_name = f"{NUM_LABELS}L"
    print(f"Nombre del experimento: {exp_name}")
    

    features_dir = os.path.join(root_dir, 'features', 'alexnet_retrained', data_flags, f'K{str(i)}')
    labels_dir = os.path.join(root_dir, 'labels', data_flags, str(i))
    output_dir = os.path.join(root_dir, "outputs", "C2AE_alexnet_retrained", data_flags, exp_name, str(i))
    model_dir = os.path.join(root_dir, 'models', 'C2AE_alexnet_retrained', data_flags, str(i))
    model_path = os.path.join(model_dir, exp_name + '.pth')


    Kfolds[i] = {
        'labels_dir': labels_dir,
        'output_dir': output_dir,
        'model_path': model_path,
        'features_dir': features_dir,
    }
    
    if not (os.path.isdir(features_dir) and os.path.isdir(labels_dir)):
        print(features_dir)
        print(labels_dir)
        raise FileNotFoundError("""
        No existen directorios de datos para el conjunto de flags seleccionado. 
        Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation.
        """)
        
    print("--Feature set encontrado en {}".format(features_dir))
    print("--Labels set encontrado en {}".format(labels_dir))
    print("")
    

    if SAVE:
        os.makedirs(output_dir, exist_ok = True)
        os.makedirs(model_dir, exist_ok = True)
        print(f"Los resultados se guardarán en: {output_dir}")
        print(f"Los modelos se guardarán en: {model_dir}")

Fold  0
Nombre del experimento: 26L
--Feature set encontrado en ../features/alexnet_retrained/base/K0
--Labels set encontrado en ../labels/base/0

Los resultados se guardarán en: ../outputs/C2AE_alexnet_retrained/base/26L/0
Los modelos se guardarán en: ../models/C2AE_alexnet_retrained/base/0
Fold  1
Nombre del experimento: 26L
--Feature set encontrado en ../features/alexnet_retrained/base/K1
--Labels set encontrado en ../labels/base/1

Los resultados se guardarán en: ../outputs/C2AE_alexnet_retrained/base/26L/1
Los modelos se guardarán en: ../models/C2AE_alexnet_retrained/base/1
Fold  2
Nombre del experimento: 26L
--Feature set encontrado en ../features/alexnet_retrained/base/K2
--Labels set encontrado en ../labels/base/2

Los resultados se guardarán en: ../outputs/C2AE_alexnet_retrained/base/26L/2
Los modelos se guardarán en: ../models/C2AE_alexnet_retrained/base/2
Fold  3
Nombre del experimento: 26L
--Feature set encontrado en ../features/alexnet_retrained/base/K3
--Labels set encont

## Entrenamiento

In [7]:
sum_f1 = 0
sum_f2 = 0
sum_recall = 0
sum_precision = 0
sum_acc = 0
sum_hl = 0
sum_emr = 0
sum_hs = 0
sum_mr1 = 0
sum_mr2 = 0
sum_mr3 = 0
sum_mr4 = 0
sum_mr5 = 0

for i in range(0, K):
    fold = Kfolds[i]
    labels_dir = fold['labels_dir']
    output_dir = fold['output_dir']
    model_path = fold['model_path']
    features_dir = fold['features_dir']
    # Carga de top labels
    train_labels = pd.read_json(os.path.join(labels_dir, 'augmented_train_df.json'), orient='index')
    
    if not os.path.isfile(os.path.join(root_dir, 'labels', f'top_{NUM_LABELS}L.pickle')):
        print(f"Creando top_labels para {NUM_LABELS} labels")
        top_labels = pruner.filter_labels(train_labels)
        pruner.set_top_labels(top_labels)
        
        save = input(f"Se creará un archivo nuevo para {len(top_labels)} labels. Desea continuar? (y/n)")
        if save == "y":
            with open(os.path.join(root_dir, 'labels', f'top_{NUM_LABELS}L.pickle'), 'wb') as f:
                pickle.dump(top_labels, f)
            print("Top labels creado con éxito")
            
        else:
            raise Exception("No se logró cargar top_labels")
            
    else: 
        print(f"Usando top_labels previamente generados para {NUM_LABELS} labels")
        with open(os.path.join(root_dir, 'labels', f'top_{NUM_LABELS}L.pickle'), 'rb') as f:
            top_labels = pickle.load(f)

    pruner = KunischPruner(NUM_LABELS)
    pruner.set_top_labels(top_labels)

    device = torch.device('cuda')



    train_labels = pd.read_json(os.path.join(labels_dir, 'augmented_train_df.json'), orient='index')
    train_labels = pruner.filter_df(train_labels)

    train_x = pd.read_json(os.path.join(features_dir, 'augmented_train_df.json'), orient='index').values
    train_y = train_labels.values

    test_labels = pd.read_json(os.path.join(labels_dir, 'test_df.json'), orient='index')
    test_labels = pruner.filter_df(test_labels)

    test_x = pd.read_json(os.path.join(features_dir, 'test_df.json'), orient='index').values
    test_y = test_labels.values
        
    train_dataset = TensorDataset(torch.tensor(train_x, 
                                               device=device, 
                                               dtype=torch.float),
                                  torch.tensor(train_y, 
                                               device=device,
                                               dtype=torch.float))
    test_dataset = TensorDataset(torch.tensor(test_x, 
                                              device=device, 
                                              dtype=torch.float), 
                                 torch.tensor(test_y, 
                                              device=device, 
                                              dtype=torch.float))

    #display(train_dataset[:][0].shape, train_dataset[:][1].shape, test_dataset[:][0].shape, test_dataset[:][1].shape)

    # Training configs.
    num_epochs = NUM_EPOCHS
    batch_size = BATCH_SIZE
    lr = 0.0001
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    # # Scene config
    feat_dim = FEATURES_DIM
    num_labels = len(top_labels)

    # Scene models.
    Fx_scene = Fx(feat_dim, fx_h_dim, fx_h_dim, latent_dim)
    Fe_scene = Fe(num_labels, fe_h_dim, latent_dim)
    Fd_scene = Fd(latent_dim, fd_h_dim, num_labels, fin_act=torch.sigmoid)

    # Initializing net.
    net = C2AE(Fx_scene, Fe_scene, Fd_scene, beta=0.5, alpha=10, emb_lambda=0.01, latent_dim=latent_dim, device=device)
    net = net.to(device)


    # Doing weight_decay here is eqiv to adding the L2 norm.
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    print("Starting training!")
    best_weights = None
    best_loss = math.inf
    patience = PATIENCE
    bad_epochs = 0

    for epoch in range(num_epochs+1): 
        # Training.
        net.train()
        loss_tracker = 0.0
        latent_loss_tracker = 0.0
        cor_loss_tracker = 0.0
        for x, y in train_dataloader:
            optimizer.zero_grad()      

            # Pass x, y to network. Retrieve both encodings, and decoding of ys encoding.
            fx_x, fe_y, fd_z = net(x, y)
            # Calc loss.
            l_loss, c_loss = net.losses(fx_x, fe_y, fd_z, y)
            # Normalize losses by batch.
            l_loss /= x.shape[0]
            c_loss /= x.shape[0]
            loss = net.beta*l_loss + net.alpha*c_loss
            loss.backward()
            optimizer.step()

            loss_tracker+=loss.item()
            latent_loss_tracker+=l_loss.item()
            cor_loss_tracker+=c_loss.item()

        # Evaluation
        net.eval()
        loss_tracker = 0.0
        latent_loss_tracker = 0.0
        cor_loss_tracker = 0.0
        acc_track = 0.0
        for x, y in test_dataloader:
            # evaluation only requires x. As its just Fd(Fx(x))
            fx_x, fe_y = net.Fx(x), net.Fe(y)
            fd_z = net.Fd(fx_x)

            l_loss, c_loss = net.losses(fx_x, fe_y, fd_z, y)
            # Normalize losses by batch.
            l_loss /= x.shape[0]
            c_loss /= x.shape[0]
            loss = net.beta*l_loss + net.alpha*c_loss

            latent_loss_tracker += l_loss.item()
            cor_loss_tracker += c_loss.item()
            loss_tracker += loss.item()
            lab_preds = torch.round(net.Fd(net.Fx(x))).cpu().detach().numpy()

        print(f"Epoch: {epoch}, Loss: {loss_tracker},  L-Loss: {latent_loss_tracker}, C-Loss: {cor_loss_tracker}")
        if cor_loss_tracker < best_loss:
            best_loss = cor_loss_tracker
            best_weights = net.state_dict()
            bad_epochs = 0
            best_epoch = epoch
        else:
            bad_epochs += 1
            if bad_epochs == patience:
                print(f"Out of patience at epoch {epoch}")
                break

    if SAVE:
        print("Guardando mejor modelo en ", model_path)
        torch.save(best_weights, model_path)

    eval_net = load_model(C2AE, model_path, 
                          Fx=Fx_scene, Fe=Fe_scene, Fd=Fd_scene, device=device).to(device)

    y_pred, y_true = get_predictions(net, [test_dataset], device)

    metrics = KunischMetrics(y_true, y_pred)
    sum_f1 += metrics.f1()
    sum_f2 += metrics.f2()
    sum_recall += metrics.recall()
    sum_precision += metrics.precision()
    sum_acc += metrics.acc()
    sum_hl += metrics.hl()
    sum_emr += metrics.emr()
    sum_hs += metrics.hs()
    sum_mr1 += metrics.mr1()
    sum_mr2 += metrics.mr2()
    sum_mr3 += metrics.mr3()
    sum_mr4 += metrics.mr4()
    sum_mr5 += metrics.mr5()

    print(f"HS fold {i}: {metrics.hs()}")

    if SAVE:
        save_df = pd.DataFrame(y_pred)
        save_df.to_csv(os.path.join(output_dir, 'predictions.csv'))
        print(f"Predicciones guardadas en {os.path.join(output_dir, 'predictions.csv')}")

        
avg_f1 = round(sum_f1/K, 4)
avg_f2 = round(sum_f2/K, 4)
avg_recall = round(sum_recall/K, 4)
avg_precision = round(sum_precision/K, 4)
avg_acc = round(sum_acc/K, 4)
avg_hl = round(sum_hl/K, 4)
avg_emr = round(sum_emr/K, 4)
avg_hs = round(sum_hs/K, 4)
avg_mr1 = round(sum_mr1/K, 4)
avg_mr2 = round(sum_mr2/K, 4)
avg_mr3 = round(sum_mr3/K, 4)
avg_mr4 = round(sum_mr4/K, 4)
avg_mr5 = round(sum_mr5/K, 4)

metadata = {
'data_flags': data_flags,
'patience': PATIENCE,
'batch_size': BATCH_SIZE,
'optimizer': (type (optimizer).__name__),
'epochs': num_epochs,
'num_labels': NUM_LABELS,
'f1': avg_f1,
'f2': avg_f2,
'recall': avg_recall,
'precision': avg_precision,
'acc': avg_acc,
'hl': avg_hl,
'emr': avg_emr,
'hs': avg_hs,
'mr1': avg_mr1,
'mr2': avg_mr2,
'mr3': avg_mr3,
'mr4': avg_mr4,
'mr5': avg_mr5
}

print("HS Final: ", avg_hs)
print("F1 Final: ", avg_f1)
print("F2 Final: ", avg_f2)
print("1MR Final: ", avg_mr1)
print("5MR Final: ", avg_mr5)

if SAVE:
    metadf = pd.DataFrame.from_dict(metadata, orient='index')
    # output_dir pero sin numero de fold
    os.makedirs(os.path.join(root_dir, 'outputs', 'C2AE_alexnet', data_flags, exp_name), exist_ok=True)
    metadf.to_csv(os.path.join(root_dir, "outputs", "C2AE_alexnet", data_flags, exp_name, 'metadata.csv'))

Usando top_labels previamente generados para 26 labels
Starting training!
Epoch: 0, Loss: 20.247953414916992,  L-Loss: 2.732123613357544, C-Loss: 1.8881890773773193
Epoch: 1, Loss: 19.669337272644043,  L-Loss: 1.7019063234329224, C-Loss: 1.8818383812904358
Epoch: 2, Loss: 19.425138473510742,  L-Loss: 1.3926793336868286, C-Loss: 1.8728798627853394
Epoch: 3, Loss: 19.267860412597656,  L-Loss: 1.2664667963981628, C-Loss: 1.863462746143341
Epoch: 4, Loss: 19.15009880065918,  L-Loss: 1.325123906135559, C-Loss: 1.848753571510315
Epoch: 5, Loss: 19.138456344604492,  L-Loss: 1.5735853910446167, C-Loss: 1.835166335105896
Epoch: 6, Loss: 19.082763671875,  L-Loss: 1.7490448355674744, C-Loss: 1.820824146270752
Epoch: 7, Loss: 18.9682674407959,  L-Loss: 1.735338568687439, C-Loss: 1.810059666633606
Epoch: 8, Loss: 18.814847946166992,  L-Loss: 1.6947866678237915, C-Loss: 1.796745479106903
Epoch: 9, Loss: 18.702113151550293,  L-Loss: 1.6335653066635132, C-Loss: 1.7885330319404602
Epoch: 10, Loss: 18.6

Epoch: 89, Loss: 17.9044246673584,  L-Loss: 1.0634883642196655, C-Loss: 1.737268090248108
Epoch: 90, Loss: 17.923744201660156,  L-Loss: 1.0399297773838043, C-Loss: 1.7403780221939087
Epoch: 91, Loss: 17.92375087738037,  L-Loss: 1.03075110912323, C-Loss: 1.7408375144004822
Epoch: 92, Loss: 17.89631175994873,  L-Loss: 1.0207402110099792, C-Loss: 1.738594114780426
Epoch: 93, Loss: 17.886780738830566,  L-Loss: 1.0063888430595398, C-Loss: 1.738358736038208
Epoch: 94, Loss: 17.891791343688965,  L-Loss: 0.9956810772418976, C-Loss: 1.739395022392273
Epoch: 95, Loss: 17.873581886291504,  L-Loss: 0.9822920560836792, C-Loss: 1.7382436394691467
Epoch: 96, Loss: 17.879032135009766,  L-Loss: 0.9695059657096863, C-Loss: 1.739427924156189
Epoch: 97, Loss: 17.86746120452881,  L-Loss: 0.9559260904788971, C-Loss: 1.7389497756958008
Epoch: 98, Loss: 17.87734031677246,  L-Loss: 0.9455108642578125, C-Loss: 1.7404584884643555
Epoch: 99, Loss: 17.866747856140137,  L-Loss: 0.9357843697071075, C-Loss: 1.7398855

Epoch: 2, Loss: 19.790912628173828,  L-Loss: 1.6210241317749023, C-Loss: 1.8980400562286377
Epoch: 3, Loss: 19.466350555419922,  L-Loss: 1.3188530802726746, C-Loss: 1.8806923627853394
Epoch: 4, Loss: 19.225903511047363,  L-Loss: 1.2204980254173279, C-Loss: 1.8615654110908508
Epoch: 5, Loss: 19.083560943603516,  L-Loss: 1.2910804748535156, C-Loss: 1.843802034854889
Epoch: 6, Loss: 18.93904399871826,  L-Loss: 1.3772651553153992, C-Loss: 1.8250410556793213
Epoch: 7, Loss: 18.820331573486328,  L-Loss: 1.4318936467170715, C-Loss: 1.8104384541511536
Epoch: 8, Loss: 18.717524528503418,  L-Loss: 1.4224196672439575, C-Loss: 1.8006314635276794
Epoch: 9, Loss: 18.651254653930664,  L-Loss: 1.4229938387870789, C-Loss: 1.793975830078125
Epoch: 10, Loss: 18.59363842010498,  L-Loss: 1.4226353168487549, C-Loss: 1.7882322072982788
Epoch: 11, Loss: 18.589316368103027,  L-Loss: 1.4564395546913147, C-Loss: 1.7861098051071167
Epoch: 12, Loss: 18.595656394958496,  L-Loss: 1.526605248451233, C-Loss: 1.7832353

Epoch: 92, Loss: 18.38786506652832,  L-Loss: 0.986676424741745, C-Loss: 1.789452612400055
Epoch: 93, Loss: 18.300519943237305,  L-Loss: 0.9740830361843109, C-Loss: 1.7813478112220764
Epoch: 94, Loss: 18.308748245239258,  L-Loss: 0.9537104070186615, C-Loss: 1.783189296722412
Epoch: 95, Loss: 18.334956169128418,  L-Loss: 0.9458479285240173, C-Loss: 1.7862032055854797
Epoch: 96, Loss: 18.3779354095459,  L-Loss: 0.9386035799980164, C-Loss: 1.7908633351325989
Epoch: 97, Loss: 18.352495193481445,  L-Loss: 0.9237476885318756, C-Loss: 1.7890621423721313
Epoch: 98, Loss: 18.27671241760254,  L-Loss: 0.9159929752349854, C-Loss: 1.7818716168403625
Epoch: 99, Loss: 18.278773307800293,  L-Loss: 0.9019007384777069, C-Loss: 1.7827823758125305
Epoch: 100, Loss: 18.38811492919922,  L-Loss: 0.8998062312602997, C-Loss: 1.7938212752342224
Epoch: 101, Loss: 18.4038724899292,  L-Loss: 0.8894714415073395, C-Loss: 1.7959136962890625
Epoch: 102, Loss: 18.376789093017578,  L-Loss: 0.8797943592071533, C-Loss: 1.7

Usando top_labels previamente generados para 26 labels
Starting training!
Epoch: 0, Loss: 22.81222629547119,  L-Loss: 7.175246477127075, C-Loss: 1.9224602580070496
Epoch: 1, Loss: 20.716912269592285,  L-Loss: 3.216932535171509, C-Loss: 1.910844624042511
Epoch: 2, Loss: 20.10086154937744,  L-Loss: 2.2094749212265015, C-Loss: 1.8996124267578125
Epoch: 3, Loss: 19.721104621887207,  L-Loss: 1.720454752445221, C-Loss: 1.8860876560211182
Epoch: 4, Loss: 19.497295379638672,  L-Loss: 1.595693290233612, C-Loss: 1.8699448704719543
Epoch: 5, Loss: 19.356765747070312,  L-Loss: 1.631691575050354, C-Loss: 1.8540920615196228
Epoch: 6, Loss: 19.221853256225586,  L-Loss: 1.682460069656372, C-Loss: 1.8380622863769531
Epoch: 7, Loss: 19.114809036254883,  L-Loss: 1.6582016348838806, C-Loss: 1.8285707831382751
Epoch: 8, Loss: 19.058125495910645,  L-Loss: 1.6395925283432007, C-Loss: 1.8238328695297241
Epoch: 9, Loss: 18.97571849822998,  L-Loss: 1.6185225248336792, C-Loss: 1.816645860671997
Epoch: 10, Loss: 

Epoch: 89, Loss: 18.114069938659668,  L-Loss: 1.121392011642456, C-Loss: 1.7553372979164124
Epoch: 90, Loss: 18.185181617736816,  L-Loss: 1.1187732815742493, C-Loss: 1.7625795006752014
Epoch: 91, Loss: 17.99306869506836,  L-Loss: 1.113682508468628, C-Loss: 1.7436226606369019
Epoch: 92, Loss: 17.996280670166016,  L-Loss: 1.0964971780776978, C-Loss: 1.7448033094406128
Epoch: 93, Loss: 18.106996536254883,  L-Loss: 1.0790977478027344, C-Loss: 1.7567447423934937
Epoch: 94, Loss: 18.170140266418457,  L-Loss: 1.0794628858566284, C-Loss: 1.7630409002304077
Epoch: 95, Loss: 18.154702186584473,  L-Loss: 1.0684436559677124, C-Loss: 1.762048065662384
Epoch: 96, Loss: 18.151122093200684,  L-Loss: 1.0593541264533997, C-Loss: 1.7621446251869202
Epoch: 97, Loss: 18.114215850830078,  L-Loss: 1.0521060228347778, C-Loss: 1.7588163018226624
Epoch: 98, Loss: 18.16058921813965,  L-Loss: 1.0433353781700134, C-Loss: 1.7638922333717346
Epoch: 99, Loss: 18.146955490112305,  L-Loss: 1.0303648710250854, C-Loss: 1

Epoch: 179, Loss: 17.88291072845459,  L-Loss: 0.42560234665870667, C-Loss: 1.7670109272003174
Epoch: 180, Loss: 17.946958541870117,  L-Loss: 0.4331011325120926, C-Loss: 1.7730408310890198
Epoch: 181, Loss: 17.946699142456055,  L-Loss: 0.40850964188575745, C-Loss: 1.7742444276809692
Epoch: 182, Loss: 17.969584465026855,  L-Loss: 0.4081389755010605, C-Loss: 1.7765514254570007
Epoch: 183, Loss: 17.94773292541504,  L-Loss: 0.427892729640007, C-Loss: 1.773378610610962
Epoch: 184, Loss: 17.868247032165527,  L-Loss: 0.39892056584358215, C-Loss: 1.7668787240982056
Epoch: 185, Loss: 17.927908897399902,  L-Loss: 0.3897229731082916, C-Loss: 1.7733046412467957
Epoch: 186, Loss: 17.87748622894287,  L-Loss: 0.3872162103652954, C-Loss: 1.7683878540992737
Epoch: 187, Loss: 17.868918418884277,  L-Loss: 0.3807680904865265, C-Loss: 1.7678533792495728
Epoch: 188, Loss: 17.89543342590332,  L-Loss: 0.3828299939632416, C-Loss: 1.7704018354415894
Epoch: 189, Loss: 17.88808822631836,  L-Loss: 0.377478525042533

Epoch: 74, Loss: 18.70551872253418,  L-Loss: 1.249152421951294, C-Loss: 1.8080942034721375
Epoch: 75, Loss: 18.48904514312744,  L-Loss: 1.2270167469978333, C-Loss: 1.7875536680221558
Epoch: 76, Loss: 18.48096466064453,  L-Loss: 1.2206991910934448, C-Loss: 1.7870614528656006
Epoch: 77, Loss: 18.517043113708496,  L-Loss: 1.206244707107544, C-Loss: 1.7913921475410461
Epoch: 78, Loss: 18.46927261352539,  L-Loss: 1.1867029070854187, C-Loss: 1.7875921130180359
Epoch: 79, Loss: 18.40500259399414,  L-Loss: 1.1755149364471436, C-Loss: 1.7817245721817017
Epoch: 80, Loss: 18.407859802246094,  L-Loss: 1.1616113781929016, C-Loss: 1.7827053666114807
Epoch: 81, Loss: 18.364352226257324,  L-Loss: 1.1440190076828003, C-Loss: 1.7792342901229858
Epoch: 82, Loss: 18.350729942321777,  L-Loss: 1.1317089200019836, C-Loss: 1.778487503528595
Epoch: 83, Loss: 18.334680557250977,  L-Loss: 1.1191979050636292, C-Loss: 1.7775080800056458
Epoch: 84, Loss: 18.36689853668213,  L-Loss: 1.1098197102546692, C-Loss: 1.781